In [1]:
import pandas as pd
import numpy as np
from time import sleep

In [2]:
#### część 1 lista wszystkich produktów

# strona uniemożliwia pobranie linków do produktów na zakładce przy pomocy BS lub Scrapy. Konieczne jest użycie Selenium.
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

d = webdriver.Chrome(executable_path='chromedriver') 

In [3]:
d.get("https://www.notino.pl/perfumy/") # link do kategorii

In [4]:
# maksymalny zakres kart na stronie
max_zakres = int(d.find_elements("xpath","//span[@data-testid='page-item']")[-1].text)

In [5]:
products = [] # lista linkow do kazdego produktu
url = 'https://www.notino.pl/perfumy/?f=0-1-55544'
for i in range(1,max_zakres+1): 
    url = url.replace(f'?f={i-1}',f'?f={i}') # kolejne zakładki powstają według takiego schematu
    d.get(url) 
    produkty = d.find_elements("xpath","//div[@data-testid='product-container']//a") # odnajduj linki do produktów
    sleep(2)
    for i in produkty:
        products.append(i.get_attribute('href')) # dopisuj linki do listy
products = list(set(products)) # unikatowe wartości
niechciane_adresy = ['https://www.notino.pl/opakowanie-prezentowe/', 'https://www.notino.pl/mobile-application/'] # zbędne adresy
for adres in niechciane_adresy:
    if adres in products:
        products.remove(adres) 

In [6]:
d.close() 

In [7]:
# zapisuje liste ze wszystkimi linkami do pliku csv, żeby nie było potrzeby za każdym razem powtarzać powyższego
df = pd.DataFrame(products)
df.dropna(inplace=True)
df.to_csv("link_list.csv", index = False)

In [8]:
#### część 2 scrapowanie poszczegolnych produktów

In [9]:
import scrapy
import json
import logging
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor

In [10]:
# format pliku wynikowego
class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('output.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [11]:
class ProductsSpider(scrapy.Spider):
    
    name = 'products'
    start_urls = products
            
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, 
        'FEED_FORMAT':'json',                                 
        'FEED_URI': 'output.json'
       ,'DOWNLOAD_DELAY': 3 # opóźnienie w sekundach
       , 'RANDOMIZE_DOWNLOAD_DELAY' : True # losowość opóźnień: (0.5,1.5)*lag
    }
    
    def parse_product(self,response):
        
        xpath_name = '//title/text()' # ścieżka do nazwy 
        xpath_EAN = '//script[@type="application/ld+json"]//text()' # ścieżka do EAN
        xpath_promo = '//div[@id = "pd-price"]/span/@content' # ścieżka do ceny promocyjnej
        xpath_regural = '//span[@data-testid = "originalPriceLineThroughWrapper"]/span/span/@content' # ścieżka do ceny regularnej
            
        name = response.xpath(xpath_name).get()[:-12]
        EAN = json.loads(response.xpath(xpath_EAN).get())['gtin13']
        price = response.xpath(xpath_promo).get()
        regural = response.xpath(xpath_regural).get()
        
        yield {
                'Name': name,
                'EAN': EAN,
                'cena': regural,
                'cena promo': price,
                'url': response.url
                }
    
    def parse(self,response):
        
        # ponownie dany produkt może mieć wiele objętości
        xpath_objetosc = '//div[@id = "pdVariantsTile"]//@href' 
        
        if response.xpath(xpath_objetosc).extract() == []: # jeżeli jest tylko jedna objętość to zescrapuj response.url
            yield scrapy.Request(response.url, self.parse_product)
            
        else: # jeśli jest wiele objętości wygeneruj ich listę i scrapuj wszystko
            for url in response.xpath(xpath_objetosc).extract():
                href = response.urljoin(url)
                yield scrapy.Request(href, self.parse_product)

In [12]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.34 Safari/537.36'
})

process.crawl(ProductsSpider)
process.start()

2023-02-03 01:42:59 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: scrapybot)
2023-02-03 01:42:59 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.12, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.9.4 (tags/v3.9.4:1f2e308, Apr  6 2021, 13:40:21) [MSC v.1928 64 bit (AMD64)], pyOpenSSL 23.0.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 39.0.0, Platform Windows-10-10.0.19041-SP0
2023-02-03 01:42:59 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/101.0.4951.34 Safari/537.36'}
2023-02-03 01:42:59 [py.warnings] WARNING: c:\users\norbert\appdata\local\programs\python\python39\lib\site-packages\scrapy\utils\request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have n

In [13]:
dfjl = pd.read_json('output.jl', lines=True, dtype = str) # wczytanie wyników
# ponownie cena promocyjna jest traktowana jako reguralna, więc przesuwam wszystko w lewo
dfjl["cena"] = np.where(dfjl['cena'] == 'None', dfjl["cena promo"], dfjl["cena"]) 
dfjl["cena promo"] =np.where(dfjl["cena promo"] == dfjl["cena"], np.nan, dfjl['cena promo'])
dfjl = dfjl[dfjl['cena'] != 'None']
dfjl=dfjl.dropna(axis=0)
dfjl['cena promo'] = dfjl['cena promo'].apply(lambda x: float(x.replace(',','.')) if type(x) == str else x)
dfjl['cena'] = dfjl['cena'].apply(lambda x: float(x.replace(',','.')) if type(x) == str else x)

In [14]:
# zapisuje do excela
dfjl.to_excel('wynik3.xlsx', sheet_name='Arkusz', index = False, freeze_panes = [1,0])